### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 8754
print(newsgroups_train.data[idx])


/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for my *own* morality, and I permit you to do 
the same for yourself.  I

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
cossim

array([0.11252759, 0.09561582, 0.17267024, ..., 0.09162675, 0.1121114 ,
       0.03334953])

In [16]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.49040531, 0.48118373, ..., 0.        , 0.        ,
       0.        ])

In [17]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 8754,  6552, 10613, ...,  6988,  6980,  9520])

In [18]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [19]:
mostsim

array([ 6552, 10613,  3616,  8726,  3902])

In [20]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.religion.misc'

In [21]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.religion.misc
talk.religion.misc
talk.religion.misc
talk.politics.mideast
talk.religion.misc


### Modelo de clasificación Naïve Bayes

In [22]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [23]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [24]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**No puedes usar la misma solución ya presentada por alguien en el foro antes que Ud. Es decir, sus 5 documentos al azar deben ser diferentes a los ya presentados, o las palabras que elija para el ejercicio 3 deben ser diferentes a las ya presentadas.**



In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import random
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [26]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',  remove=('headers', 'footers', 'quotes'))
# remove elimina partes no útiles de los correos: encabezados, pies y citas.

# accedemos a los textos y las etiquetas
train_docs = newsgroups_train.data
test_docs  = newsgroups_test.data

train_labels = newsgroups_train.target
test_labels  = newsgroups_test.target

label_names = newsgroups_train.target_names  # las categorías (20 temas)

print(f"Total documentos de entrenamiento: {len(train_docs)}")
print(f"Total documentos de prueba: {len(test_docs)}")
print(f"Total categorías: {len(label_names)}")




Total documentos de entrenamiento: 11314
Total documentos de prueba: 7532
Total categorías: 20


In [39]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) #para que no me explote la compu
# Crea un vectorizador TF-IDF para transformar texto en números
X_train = vectorizer.fit_transform(train_docs)
X_test = vectorizer.transform(test_docs)

y_train = newsgroups_train.target
y_test  = newsgroups_test.target
print(f"Matriz TF-IDF (train) shape: {X_train.shape}")
print(f"Matriz TF-IDF (test) shape:  {X_test.shape}")


Matriz TF-IDF (train) shape: (11314, 5000)
Matriz TF-IDF (test) shape:  (7532, 5000)


In [28]:

SEED = 2477
rng = np.random.default_rng(SEED) #le pasamos un numero aleatorio en seed para que genere los 5 numeros aleatorios que necesitamos
n = X_test.shape[0]

idx_pool = rng.permutation(n)
pick = np.sort(idx_pool[:5])   # tomamos 5 documentos aleatorios del test

print("\nDocumentos elegidos al azar del set de prueba:")
for i in pick:
    print(f"  Doc {i} , Categoría: {label_names[test_labels[i]]}")


Documentos elegidos al azar del set de prueba:
  Doc 467 , Categoría: rec.sport.baseball
  Doc 4008 , Categoría: sci.med
  Doc 4545 , Categoría: misc.forsale
  Doc 5245 , Categoría: rec.autos
  Doc 7249 , Categoría: rec.sport.hockey


In [45]:
for i in pick:
    print("\n" + "="*140)
    print(f" Documento TEST {i} | Categoría: {label_names[test_labels[i]]}")
    print("°"*90)
    print(test_docs[i][:400].replace("\n", " ") + "...")  # fragmento del texto

    # calculamos similitud del documento test i con todos los documentos train
    cossim = cosine_similarity(X_test[i], X_train)[0]

    # ordenamos índices de mayor a menor similitud
    most_similar_idx = np.argsort(-cossim)

    # top 5 más similares en train (excluyendo el mismo si estuviera)
    top5 = most_similar_idx[:5]

    print("\n** Top 5 documentos más similares en TRAIN:**")
    for j in top5:
        print(f"\n  → Doc TRAIN {j} | Similitud: {cossim[j]:.3f} | Categoría: {label_names[train_labels[j]]}")
        print(f"    Texto: {train_docs[j][:200].replace(chr(10), ' ')}...")  # primeros 200 caracteres



 Documento TEST 467 | Categoría: rec.sport.baseball
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
To encourage the great tradition of Red Sox negativism, I am having a  contest to predict the magnitude of the Sox' fall from their current  heights.  You must decide first, whether the Sox will be at .500 again at any time during this year.  Then you must predict either:  (1) Their record the first time they're at .500, if they are, or  (2) Their final record, if they stay above .500 the rest of ...

** Top 5 documentos más similares en TRAIN:**

  → Doc TRAIN 7974 | Similitud: 0.462 | Categoría: rec.sport.baseball
    Texto: Today, Frank Viola and rest of pitcher staff of Boston Red Sox shutout Chicago White Sox 4-0.  It is Red Sox 9th win of this season....

  → Doc TRAIN 7953 | Similitud: 0.424 | Categoría: rec.sport.baseball
    Texto:  As long as the Yankees are in the same division the Red Sox will play better than .500 baseball.  Or the


## el documento 467 habla de baseball y sus similitudes tienen sentido porque son todas de la categoria re.sport.baseball

-----------------------------------------------------


## el 4008 habla de computacion y electronica y los documentos relacionados son de los mismos temas asi que tambien esta correcto

-----------------------------------------------------
## el 4545 tiene distintas categorias entre los docs mas similares, pero todos teinen en comun quje hablan sobre IBM asis que lo doy por correcto

--------------------------------------------------------------
## el 5245 esta en la categoria rec.autos, y casi todos los articulos relacionados parecen tener que ver en tematica mas alla de su categoria, pero hay algunos que parecen hablar de cosas totalmente diferentes

--------------------------------------------------------------
## en el 7249 todos los documentos son similares asi que tiene sentido




**2**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [33]:
X_train_transposed = X_train.T
print(f"Matriz término-documento: {X_train_transposed.shape}")

Matriz término-documento: (1000, 11314)


In [35]:
selected_words = ['car', 'bible', 'price', 'health', 'space']
selected_words = [w.lower() for w in selected_words]  # TF-IDF convierte todo a minúsculas

# Verificamos que las palabras están realmente en el vocabulario porque al acotarlo muchas que parecen obvias no estan
for w in selected_words:
    if w in selected_words:
        print(f"'{w}' está en el vocabulario")
    else:
        print(f"'{w}' NO está en el vocabulario")


'car' está en el vocabulario
'bible' está en el vocabulario
'price' está en el vocabulario
'health' está en el vocabulario
'space' está en el vocabulario


In [44]:


# Obtenemos vocabulario
terms = vectorizer.get_feature_names_out()

# Transponemos la matriz
X_words = X_train.T

# Calculamos similitud coseno entre palabras
similarities_terms = cosine_similarity(X_words.toarray())

# Palabras manualmente seleccionadas
selected_words = ['car', 'bible', 'price', 'health', 'space']

# Buscamos los índices de las palabras
indices = [np.where(terms == w)[0][0] for w in selected_words]
print("Indices de las palabras seleccionadas:", indices)

#de cada palabra buscamos las 5 mas similares

for word in selected_words:
   if word in terms:
    # Índice de la palabra en vocabulario
    idx = np.where(terms == word)[0][0]

    # Similitud con todas las palabras
    sim_scores = list(enumerate(similarities_terms[idx]))

    # Ordenar de mayor a menor
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Tomamos las 5 más similares (excluyendo la misma palabra)
    top5 = [terms[j] for j, score in sim_scores[1:6]]
   else:
         print(f"La palabra '{word}' no está en el vocabulario")
    # Mostrar resultados
   print("="*80)
   print(f" Palabra: '{word}'")
   print("Palabras más similares:", top5)


Indices de las palabras seleccionadas: [np.int64(900), np.int64(724), np.int64(3491), np.int64(2159), np.int64(4221)]
 Palabra: 'car'
Palabras más similares: ['cars', 'dealer', 'owner', 'engine', 'mileage']
 Palabra: 'bible'
Palabras más similares: ['god', 'christians', 'jesus', 'christian', 'interpretation']
 Palabra: 'price'
Palabras más similares: ['prices', 'sale', 'new', 'brand', 'dealer']
 Palabra: 'health'
Palabras más similares: ['care', 'insurance', 'hicnet', 'medicine', 'medical']
 Palabra: 'space'
Palabras más similares: ['nasa', 'shuttle', 'exploration', 'launch', 'station']



**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [40]:
from sklearn.naive_bayes import MultinomialNB  #importamos le modelo naive bayes

clf = MultinomialNB() #creamos una instancia de clasificador
clf.fit(X_train, y_train) #entrena al modelo

tfidfvect = TfidfVectorizer(stop_words='english', max_features=5000) #vectorizamos y limite de 5000 palabras en el vocabulario
X_train = tfidfvect.fit_transform(newsgroups_train.data)  # Fit y transform sobre train
X_test = tfidfvect.transform(newsgroups_test.data)        # Solo transform sobre test

y_pred = clf.predict(X_test)
# 'macro' = promedio de F1-score por clase (útil con desbalance de clases)
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f1_macro)

0.6243150711044391


In [41]:
models = {
    "MultinomialNB": MultinomialNB(alpha=0.5),
    "ComplementNB": ComplementNB(alpha=0.5)
}


for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"{name} → F1 macro: {f1:.4f}")

    #este codigo nos muestra el entenamiento y evaluacion de dos modelos naive bayes
    # lo hice manualmente porque se me complicaba hacer un bucle que pruebe  hasta dar con la  mejor configuracion

MultinomialNB → F1 macro: 0.6337
ComplementNB → F1 macro: 0.6335


# En este trabajo los pasos que seguimos fueron:

# 1- carga del dataset

# 2-vectorizacion

# 3-entrenamiento del modelo naive bayes

# 4-evaluacion del desempeño

# fue un trabajo bastante desafiante en el que aprendi bastantes cosas nuevas y vi como se relacionan con lo anterior que ya venimos viendo, el modelo fue un exito con un f1 macro de 0.624 antes de los ajustes y logrando 0.633 despues de algunas modificaciones, lo que nos da una buena probabilidad de lograr una correcta clasificacion de las palabras y los documentos